In [1]:
import json
import time
import os
import re

from modules.preprocessing_protocols import preprocess_text
from modules.extract_results import extract_results_to_txt_file, display_results_by_subdistribution,\
                                    display_results_by_named_entity, display_confusion_matrix

from estnltk import Text
from estnltk.taggers import NerTagger
from estnltk.taggers import WordLevelNerTagger
from estnltk.converters import text_to_json
from estnltk.converters import json_to_text
from estnltk.layer_operations import flatten

from estnltk.taggers.estner.ner_trainer import NerTrainer
from estnltk.taggers.estner.model_storage_util import ModelStorageUtil

In [2]:
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

These files don't work because the protocols are written in a different language, which the goldstandard didn't recognise, hence have no goldstandard tags.

In [3]:
files_not_working = ['J2rva_Tyri_V22tsa_id22177_1911a.json', \
                     'J2rva_Tyri_V22tsa_id18538_1894a.json', \
                     'J2rva_Tyri_V22tsa_id22155_1911a.json', \
                     'Saare_Kihelkonna_Kotlandi_id18845_1865a.json', \
                     'P2rnu_Halliste_Abja_id257_1844a.json', \
                     'Saare_Kaarma_Loona_id7575_1899a.json', \
                     'J2rva_Tyri_V22tsa_id22266_1913a.json', \
                     'J2rva_Tyri_V22tsa_id22178_1912a.json']

models = ['model_morph_with_lemmas',\
          'model_morph_with_lemmas_and_sentences',\
          'model_morph_with_lemmas_and_sentences_and_gazzetteer']

In [4]:
files = {}

with open(os.path.join('..', 'data', 'divided_corpus.txt'), 'r', encoding = 'UTF-8') as f:
    txt = f.readlines()

for fileName in txt:
    file, subdistribution = fileName.split(":")
    files[file] = subdistribution.rstrip("\n")

In [5]:
for model_directory in models:
    for subdistribution in [1, 2, 3, 4, 5]:
        training_subdistributions = []
        for y in [1, 2, 3, 4, 5]:
            if y == subdistribution:
                subdistribution_for_testing = y
            else:
                training_subdistributions.append(y)

        # Getting the filenames to be trained on from the files dictionary.
        filenames = {key: value for key, value in files.items() if int(value) in training_subdistributions}

        # Creating training_texts from the aforementioned filenames.
        print("Valmistan ette treenimistekste.")
        start = time.time()
        training_texts = []
        for filename in filenames:
            with open(os.path.join('..', 'data', 'vallakohtufailid-json-flattened', filename), 'r', encoding='UTF-8') as file:
                if filename in files_not_working:
                    continue
                else:
                    training_texts.append(preprocess_text(json_to_text(file.read())))
        print(f"Treenimistekstid ette valmistatud {time.time() - start} sekundiga.")
        # Setting up the trainer and training.
        print("Alustan NerTaggeri treenimist.")
        start = time.time()
        new_model_dir = os.path.join('models', model_directory)
        modelUtil = ModelStorageUtil( new_model_dir )
        nersettings = modelUtil.load_settings()
        trainer = NerTrainer(nersettings)
        trainer.train( training_texts, layer='gold_wordner', model_dir=new_model_dir )
        print(f"NerTagger treenitud {time.time() - start} sekundiga.")

        # Setting up the new trained nertagger and defining layers to be removed later on.
        nertagger = NerTagger(model_dir = new_model_dir)
        removed_layers = ['sentences', 'morph_analysis', 'compound_tokens', 'ner', 'words', 'tokens']

        # Tagging the files using the new nertagger.
        print("\n\nAlustan failide märgendamist.")
        start = time.time()
        testing_files = {key: value for key, value in files.items() if int(value) == subdistribution_for_testing}
        for file in testing_files:
            with open(find(file.replace(".json", ".txt"), os.path.join('..', 'data', 'vallakohtufailid')), 'r', encoding='UTF-8') as f:
                text = f.read()
                if file == "Tartu_V6nnu_Ahja_id3502_1882a.txt":
                    text = text.replace('..', '. .')
                text = preprocess_text(Text(text))
                nertagger.tag(text)
                text.add_layer(flatten(text['ner'], 'flat_ner'))

                for x in removed_layers:
                    text.pop_layer(x)

                text_to_json(text, file=os.path.join(new_model_dir, 'vallakohtufailid-trained-nertagger', file))
                print(f'Märgendatud fail {file}')
        print(f"Failid märgendatud {time.time() - start} sekundiga.")
print("Programm on lõpetanud oma töö.")

Valmistan ette treenimistekste.
Treenimistekstid ette valmistatud 101.83562016487122 sekundiga.
Alustan NerTaggeri treenimist.
(!) Warning: Location of the new "settings.py" is the same one as the old one. Model's settings are not copied.
Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 33508
Seconds required: 0.198

Stochastic Gradient Descent (SGD)
c2: 0.001000
max_iterations: 1000
period: 10
delta: 0.000001

Calibrating the learning rate (eta)
calibration.eta: 0.100000
calibration.rate: 2.000000
calibration.samples: 1000
calibration.candidates: 10
calibration.max_trials: 20
Initial loss: 30999.990087
Trial #1 (eta = 0.100000): 3844.097079
Trial #2 (eta = 0.200000): 5400.307219
Trial #3 (eta = 0.400000): 10056.158513
Trial #4 (eta = 0.800000): 17363.458636
Trial #5 (eta = 1.600000): 32051.440575 (worse)
Trial #6 (eta = 0.050000): 3553.442400
Tria

***** Epoch #37 *****
Loss: 3819.051097
Improvement ratio: 0.228300
Feature L2-norm: 134.492301
Learning rate (eta): 0.049816
Total number of feature updates: 600991
Seconds required for this iteration: 0.197

***** Epoch #38 *****
Loss: 3720.993324
Improvement ratio: 0.217609
Feature L2-norm: 135.716417
Learning rate (eta): 0.049811
Total number of feature updates: 617234
Seconds required for this iteration: 0.196

***** Epoch #39 *****
Loss: 3808.643347
Improvement ratio: 0.178336
Feature L2-norm: 136.959613
Learning rate (eta): 0.049806
Total number of feature updates: 633477
Seconds required for this iteration: 0.197

***** Epoch #40 *****
Loss: 3695.140470
Improvement ratio: 0.165120
Feature L2-norm: 138.151484
Learning rate (eta): 0.049801
Total number of feature updates: 649720
Seconds required for this iteration: 0.201

***** Epoch #41 *****
Loss: 3639.711516
Improvement ratio: 0.141549
Feature L2-norm: 139.300715
Learning rate (eta): 0.049796
Total number of feature updates: 6

Märgendatud fail J2rva_Tyri_V22tsa_id18538_1894a.json
Märgendatud fail V6ru_R6uge_Saaluse_id11478_1879a.json
Märgendatud fail Tartu_Kodavere_Pala_id22839_1872a.json
Märgendatud fail Saare_Kihelkonna_Lymanda_id790_1868a.json
Märgendatud fail Harju_Rapla_Rapla_id20935_1870a.json
Märgendatud fail Tartu_V6nnu_Ahja_id23907_1897a.json
Märgendatud fail Harju_J6el2htme_J6el2htme_id8152_1888a.json
Märgendatud fail L22ne_Pyhalepa_K2rdla_id23487_1873a.json
Märgendatud fail J2rva_Tyri_V22tsa_id17593_1888a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id11023_1880a.json
Märgendatud fail Harju_Kose_Palvere_id19503_1884a.json
Märgendatud fail J2rva_Anna_Eivere_id1936_1872a.json
Märgendatud fail Tartu_Kodavere_Ranna_id15116_1863a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id7763_1879a.json
Märgendatud fail J2rva_Tyri_Kirna_id25043_1881a.json
Märgendatud fail Tartu_V6nnu_Ahja_id14888_1882a.json
Märgendatud fail L22ne_Kullamaa_Kuij6e_id15484_1877a.json
Märgendatud fail Tartu_Torma_Avinurme_id2512_1

Märgendatud fail Tartu_V6nnu_Ahja_id20664_1889a.json
Märgendatud fail Harju_Rapla_Rapla_id15629_1867a.json
Märgendatud fail Tartu_V6nnu_Ahja_id16115_1883a.json
Märgendatud fail V6ru_Urvaste_Vaabina_id798_1876a.json
Märgendatud fail V6ru_R2pina_Kahkva_id7319_1888a.json
Märgendatud fail Tartu_Kodavere_Pala_id22626_1871a.json
Märgendatud fail Harju_Kose_Palvere_id15360_1880a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id6838_1879a.json
Märgendatud fail Viljandi_P6ltsamaa_Pajusi_id2106_1870a.json
Märgendatud fail V6ru_R6uge_Saaluse_id9979_1879a.json
Märgendatud fail Tartu_V6nnu_Ahja_id23400_1893a.json
Märgendatud fail J2rva_Anna_Purdi_id18906_1870a.json
Märgendatud fail Harju_Juuru_Kaiu_id13544_1884a.json
Märgendatud fail Tartu_V6nnu_Ahja_id13151_1876a.json
Märgendatud fail Harju_Kose_Palvere_id25267_1873a.json
Märgendatud fail Viljandi_P6ltsamaa_Uue-P6ltsamaa_id8744_1854a.json
Märgendatud fail Harju_Kose_Triigi_id10831_1870a.json
Märgendatud fail J2rva_Tyri_Kirna_id23927_1875a.json
Mär

***** Epoch #30 *****
Loss: 5447.796456
Improvement ratio: 0.204865
Feature L2-norm: 93.256420
Learning rate (eta): 0.024963
Total number of feature updates: 486030
Seconds required for this iteration: 0.200

***** Epoch #31 *****
Loss: 5366.295667
Improvement ratio: 0.209356
Feature L2-norm: 94.428310
Learning rate (eta): 0.024961
Total number of feature updates: 502231
Seconds required for this iteration: 0.202

***** Epoch #32 *****
Loss: 5418.160146
Improvement ratio: 0.172952
Feature L2-norm: 95.610397
Learning rate (eta): 0.024960
Total number of feature updates: 518432
Seconds required for this iteration: 0.203

***** Epoch #33 *****
Loss: 5247.542099
Improvement ratio: 0.173347
Feature L2-norm: 96.715048
Learning rate (eta): 0.024959
Total number of feature updates: 534633
Seconds required for this iteration: 0.199

***** Epoch #34 *****
Loss: 5145.497763
Improvement ratio: 0.174268
Feature L2-norm: 97.823129
Learning rate (eta): 0.024958
Total number of feature updates: 550834

...

Märgendatud fail J2rva_Ambla_Ambla_id6188_1889a.json
Märgendatud fail J2rva_Ambla_Ambla_id6411_1882a.json
Märgendatud fail Viljandi_P6ltsamaa_Adavere_id22490_1896a.json
Märgendatud fail L22ne_Kullamaa_Piirsalu_id14765_1882a.json
Märgendatud fail V6ru_R2pina_R2pina_id1978_1862a.json
Märgendatud fail L22ne_Kullamaa_Piirsalu_id7490_1884a.json
Märgendatud fail Viljandi_Paistu_Holstre_id6601_1827a.json
Märgendatud fail L22ne_Kullamaa_Sooniste_id3738_1884a.json
Märgendatud fail L22ne_Emmaste_Emmaste_id15405_1896a.json
Märgendatud fail J2rva_Peetri_V2ike-Kareda_id21174_1874a.json
Märgendatud fail Harju_Hageri_Kohila_id11667_1879a.json
Märgendatud fail Harju_Juuru_Juuru_id14480_1877a.json
Märgendatud fail L22ne_Kullamaa_K22nda_id18344_1871a.json
Märgendatud fail Harju_Hageri_Kohila_id24217_1875a.json
Märgendatud fail Tartu_V6nnu_Ahja_id17527_1885a.json
Märgendatud fail P2rnu_Tori_Sindi_id11974_1885a.json
Märgendatud fail Viljandi_P6ltsamaa_Adavere_id17828_1894a.json
Märgendatud fail Tartu_Koda

Märgendatud fail L22ne_Kullamaa_Sooniste_id3686_1883a.json
Märgendatud fail Harju_Juuru_Kaiu_id12588_1883a.json
Märgendatud fail Tartu_Otep22_Pyhaj2rve_id11890_1884a.json
Märgendatud fail Tartu_V6nnu_Ahja_id23070_1891a.json
Märgendatud fail P2rnu_Tori_Sindi_id20040_1836a.json
Märgendatud fail Tartu_Torma_Avinurme_id21181_1871a.json
Märgendatud fail L22ne_Pyhalepa_K2rdla_id22386_1867a.json
Märgendatud fail Tartu_V6nnu_Ahja_id21023_1889a.json
Märgendatud fail Tartu_Kodavere_Pala_id20578_1867a.json
Märgendatud fail V6ru_Vastseliina_Misso_id21929_1883a.json
Märgendatud fail L22ne_Ridala_Sinalepa_id25579_1890a.json
Märgendatud fail J2rva_Tyri_Kirna_id25219_1882a.json
Märgendatud fail J2rva_Tyri_S2revere_id11686_1874a.json
Märgendatud fail Tartu_V6nnu_Ahja_id18911_1887a.json
Märgendatud fail Saare_P8ide_Laimjala_id10240_1910a.json
Märgendatud fail Tartu_Torma_Avinurme_id17369_1871a.json
Märgendatud fail Harju_Keila_Keila_id13018_1890a.json
Märgendatud fail V6ru_Vastseliina_Misso_id21891_1883

***** Epoch #20 *****
Loss: 4038.420687
Improvement ratio: 0.478970
Feature L2-norm: 103.082622
Learning rate (eta): 0.049900
Total number of feature updates: 312280
Seconds required for this iteration: 0.306

***** Epoch #21 *****
Loss: 3954.235660
Improvement ratio: 0.412986
Feature L2-norm: 104.949755
Learning rate (eta): 0.049895
Total number of feature updates: 327894
Seconds required for this iteration: 0.309

***** Epoch #22 *****
Loss: 3780.927412
Improvement ratio: 0.439747
Feature L2-norm: 106.695646
Learning rate (eta): 0.049890
Total number of feature updates: 343508
Seconds required for this iteration: 0.305

***** Epoch #23 *****
Loss: 3739.781962
Improvement ratio: 0.380849
Feature L2-norm: 108.385567
Learning rate (eta): 0.049885
Total number of feature updates: 359122
Seconds required for this iteration: 0.307

***** Epoch #24 *****
Loss: 3641.338679
Improvement ratio: 0.366084
Feature L2-norm: 110.028933
Learning rate (eta): 0.049880
Total number of feature updates: 3

Märgendatud fail Tartu_Kodavere_Ranna_id19673_1865a.json
Märgendatud fail J2rva_Tyri_S2revere_id9244_1885a.json
Märgendatud fail P2rnu_P2rnu-Elisabethi_Sauga_id22818_1882a.json
Märgendatud fail Tartu_V6nnu_Ahja_id16349_1884a.json
Märgendatud fail Harju_Kuusalu_Kolga_id11902_1888a.json
Märgendatud fail J2rva_Peetri_V2ike-Kareda_id22150_1879a.json
Märgendatud fail Tartu_Kodavere_Pala_id22870_1872a.json
Märgendatud fail Tartu_V6nnu_Ahja_id13565_1881a.json
Märgendatud fail Harju_Juuru_Juuru_id17866_1869a.json
Märgendatud fail Harju_Juuru_Kaiu_id931_1912a.json
Märgendatud fail J2rva_Anna_Eivere_id6607_1880a.json
Märgendatud fail L22ne_Kullamaa_Piirsalu_id16723_1888a.json
Märgendatud fail J2rva_Tyri_V22tsa_id17604_1888a.json
Märgendatud fail Tartu_V6nnu_Ahja_id14708_1882a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id9914_1880a.json
Märgendatud fail L22ne_Pyhalepa_K2rdla_id23461_1873a.json
Märgendatud fail Tartu_V6nnu_Ahja_id21842_1868a.json
Märgendatud fail Harju_J6el2htme_J6el2htme_id76

Märgendatud fail Tartu_V6nnu_Ahja_id14731_1882a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id23690_1866a.json
Märgendatud fail Harju_Kose_Palvere_id17074_1882a.json
Märgendatud fail Viljandi_Paistu_Holstre_id515_1843a.json
Märgendatud fail Tartu_Otep22_Pyhaj2rve_id11749_1885a.json
Märgendatud fail Harju_Harju-Madise_Padise-Kloostri_id3283_1885a.json
Märgendatud fail Tartu_V6nnu_Ahja_id15058_1883a.json
Märgendatud fail Tartu_Torma_Avinurme_id14255_1903a.json
Märgendatud fail Tartu_Kodavere_Ranna_id15105_1863a.json
Märgendatud fail Harju_Hageri_Kohila_id10294_1868a.json
Märgendatud fail Tartu_Kodavere_Pala_id16727_1855a.json
Märgendatud fail J2rva_Peetri_V2ike-Kareda_id22431_1880a.json
Märgendatud fail Saare_Kihelkonna_Kotlandi_id21549_1869a.json
Märgendatud fail J2rva_Tyri_V22tsa_id18782_1897a.json
Märgendatud fail J2rva_Tyri_V22tsa_id16955_1887a.json
Märgendatud fail J2rva_J2rva-Jaani_Einmanni_id9007_1868a.json
Märgendatud fail Tartu_V6nnu_Ahja_id23568_1895a.json
Märgendatud fail T

In [6]:
for model_directory in models:
    extract_results_to_txt_file(model_directory, files)

Tulemuste ammutamine on lõpetatud.
Tulemuste ammutamine on lõpetatud.
Tulemuste ammutamine on lõpetatud.
Tulemuste ammutamine on lõpetatud.
Tulemuste ammutamine on lõpetatud.
